In [1]:
import torch
from torch import nn
import torch.optim as optim

# パラメータ更新
注：事前に[自動微分](./AutoGrad.ipynb)を理解してから、本章を学習したほうが良い<br>
- `torch.optim` : SGDなどの複数のオプティマイザが含まれていて、逆伝搬の際に`Parameter`
の重みを更新する<br>
最適化手法には以下の種類がある
    - SGD
    - Adagrad
    - RMSprop
    - Adadelta
    - Adam
    - AdamW

今回は一番シンプルなSGDについて説明する

## SGD : Stochastic Gradient Descentとは
確率的勾配降下法と呼ばれ、パラメータの更新手法のことである
### 勾配降下法
1. 適当に初期点を決める
2. 現在の地点における降下方向を計算する<br>
降下方向：$-\frac{\partial L}{\partial w}$
3. 降下方向に進む<br>
$w = w - \eta \frac{\partial L}{\partial w}$<br>
$b = b - \eta \frac{\partial L}{\partial b}$<br>
2 に戻り、値が変化しなくなるまで繰り返す

### 確率的勾配降下法
勾配降下法の場合、データ数$N$が大きくなると、$\frac{\partial L}{\partial w}$などの計算が非常に大変になる<br>
そこで、すべてのデータを使用するのではなく、ランダムに選んだ一部のデータのみを使用
- `optim.SGD(params, lr, momentum=0, dampening=0, weight_decay=0, nesterov=False, maximize=False, foreach=None)`
    - params : 最適化するパラメータ
    - lr : 学習率
- `optimizer.step()` : 最適化ステップ（パラメータ更新）を実行している<br>

In [2]:
x = torch.tensor([1, 2, 3, 4, 5]).unsqueeze(1).float()
t = torch.tensor([5, 8, 11, 14, 17]).unsqueeze(1).float()

linear = nn.Linear(1, 1)
MSE = nn.MSELoss()
# 最適化手法の指定
optimizer = optim.SGD(linear.parameters(), lr = 1e-3)

# weight, biasの初期値を確認
print(f"Initial weight : {linear.weight.item()}")
print(f"Initial bias   : {linear.bias.item()}")

y = linear(x)
L = MSE(y, t)

L.backward()
optimizer.step()
print("-------------------")
print(f"Update weight  : {linear.weight.item()}")
print(f"Update bias    : {linear.bias.item()}")

Initial weight : 0.13374245166778564
Initial bias   : -0.08374321460723877
-------------------
Update weight  : 0.20930258929729462
Update bias    : -0.062378183007240295


# パラメータ推定
`optimizer.zero_grad()` : PyTorchの仕様で勾配（$\frac{\partial L}{\partial w}$や$\frac{\partial L}{\partial b}$）が既存の値に加算されるので、勾配値をリセットするために使用<br>
今回の場合、$x = [1, 2, 3, 4, 5]$で$t = [5, 8, 11, 14, 17]$なので、$y = 3x + 2$となるとき、$y$と$t$の差が最小になる<br>
推定結果は$weight$・$bias$ともに、最適化されている（差が最小になる値に変化している）のが確認できる

In [3]:
x = torch.tensor([1, 2, 3, 4, 5]).unsqueeze(1).float()
t = torch.tensor([5, 8, 11, 14, 17]).unsqueeze(1).float()

linear = nn.Linear(1, 1)
MSE = nn.MSELoss()
optimizer = optim.SGD(linear.parameters(), lr = 1e-3)

for epoch in range(10000):
    y = linear(x)
    L = MSE(y, t)
    optimizer.zero_grad()
    L.backward()
    optimizer.step()

print(f"Opt weight     : {linear.weight.item()}")
print(f"Opt bias       : {linear.bias.item()}")

Opt weight     : 3.017526865005493
Opt bias       : 1.9367316961288452
